<a href="https://colab.research.google.com/github/alaasalmo/CodeDeployGitHubDemo/blob/master/CIND840_Lab_3_Other_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIND840 Practical Approaches in Machine Learning
# Lab 3 - Solutions

# Preparation

In [1]:
# Install and import required libraries
# pyfpgrowth(https://fp-growth.readthedocs.io/en/latest/)
!pip install pyfpgrowth # ! is needed for Google Colab, it is optional for GCP
import os
import pyfpgrowth
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5490 sha256=33ae6135aeb37fe12d954374feee002acb20a2678ae246094f454fdd01b2943a
  Stored in directory: /root/.cache/pip/wheels/dc/18/5b/4a113996892937d01f5bfb727710d0dc569cfce1326014f8d0
Successfully built pyfpgrowth


# The Dataset
[Online Retail](http://archive.ics.uci.edu/ml/datasets/online+retail#) contains all the transactions occurring between 01-12-2010 and 09-12-2011 for a UK-based and registered non-store online retail.

## Dictionary
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

## Solution 1
Download the Online Retail data from given link and read it as Pandas DataFrame. Display the first few rows of the data. Examine its properties such as rows, columns, any missing values and columns names. Read the data dictionary and confirm your understanding of each column.

In [2]:
# Following shell commands will download the data and list contents of directory.
!wget -q -N 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
!ls
# For convenience the file name is changed.
!mv 'Online Retail.xlsx' OnlineRetail.xlsx

'Online Retail.xlsx'   sample_data


In [4]:
# read_excel function reads excel sheet as a DataFrame
dataset = pd.read_excel('OnlineRetail.xlsx')

In [ ]:
dataset.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# check shape (rows, columns) of dataset
print(dataset.shape)
# check null (missing) values in individual columns
print(dataset.isnull().sum())
# check null (missing) values in the dataset
print(dataset.isnull().sum().sum()) # 136534
# The columns names
print(dataset.columns)

(541909, 8)
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
136534
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


In [ ]:
# If InvoiceNo starts with letter 'c', it indicates a cancellation.
dataset_x = dataset[~ dataset['InvoiceNo'].str.startswith('C', na=False)] # filter 9288 cancelled or NaN transations
# Count cancelled invoices
dataset_x['InvoiceNo'].count() # 532621 left out of 541909

np.int64(532621)

In [ ]:
print(dataset['InvoiceNo'].str.startswith('C', na=False).count())

541909


## Solution 2
InvoiceNumber column provides a unique id to each transaction. Some invoice number start with 'c' indicating cancelled transactions. Filter cancelled transactions from your dataset. Then, generate frequency count of items in each transaction. Create a list of unique Invoices that have more than 5 and less than 10 items. Final data must not include cancelled transactions and transactions with less than 5 or more than 10 items.

In [ ]:
# filter transaction where items bought are less than 5
min_items_in_transaction = 5
max_items_in_transaction = 10
InvoiceNo_Count = dataset_x['InvoiceNo'].value_counts() # 25900 transactions
InvoiceNo_List = list(set(InvoiceNo_Count[InvoiceNo_Count.between(min_items_in_transaction, max_items_in_transaction)].index.tolist()))
dataset_x2 = dataset_x[dataset_x['InvoiceNo'].isin(InvoiceNo_List)]
#print('transactions in final dataset', dataset_x2.shape[0])
#print('Invoices', len(InvoiceNo_List))
#dataset_x2.head(5)
print(dataset_x2.count())

InvoiceNo      26975
StockCode      26975
Description    26975
Quantity       26975
InvoiceDate    26975
UnitPrice      26975
CustomerID     26187
Country        26975
dtype: int64


## Solution 3
Transform your filtered dataset into two python dictionaries. Each item of the first dictionary StockCode2Alias_Dict will have StockCode as key and a unique-id as value. Each item of the second dictionary Alias2StockCode_Dict will have unique-id as key and StockCode and Description as value. A StockCode can have more than one descriptions. Keep the first description of each StockCode.

In [ ]:
# print(dataset_x2[dataset_x2['StockCode'] == '85123A']['Description'].unique())
StockCode2Alias_Dict = {}
Alias2StockCode_Dict = {}
StockCodes = list(dataset_x2['StockCode'].unique())
print(len(StockCodes))
for c in StockCodes:
  if c not in StockCode2Alias_Dict.keys():
    # add item to StockCode2Alias_Dict
    StockCode2Alias_Dict[c] = len(StockCode2Alias_Dict) # StockCode_Num_Alias
    # keep the first description
    desc = dataset_x2[dataset_x2['StockCode'] == c]['Description'].unique()[0]
    # add item to Alias2StockCode_Dict
    Alias2StockCode_Dict[StockCode2Alias_Dict[c]] = (c, desc) # StockCode_Num_Alias
# print(StockCode2Alias_Dict)
print(len(Alias2StockCode_Dict))


2687
2687


## Solution 4
Create a list and name it as transactions. Append the items from InvoiceNo_List, created in Question 2, to it. Each item in transactions contains list of StockCodes of the items bought.

In [ ]:
transactions = []
for invoice_no in InvoiceNo_List:
    items = dataset_x2[dataset_x2['InvoiceNo'] == invoice_no]['StockCode'].to_list()
    transactions.append([StockCode2Alias_Dict[i] for i in items])
print('transactions_db size', len(transactions))
print(transactions[0:9])

## Solution 5
Extract Association Rules by using Frequent Patterm Mining algorithm (FPGrowth Tree). The transactions list is going to be the input to the algorithm. Print the extracted rules with the help of Alias2StockCode_Dict created in Question 3.

In [ ]:
patterns = pyfpgrowth.find_frequent_patterns(transactions, 5) # try different support threshold values

In [ ]:
rules = pyfpgrowth.generate_association_rules(patterns, 1) # try different probability threshold values
# rules are a dictionary of antecedents and consequents.

In [ ]:
for antecedents, consequents in rules.items():
    antec_list =[]
    conseq_list =[]
    for a in antecedents:
        antec_list.append(Alias2StockCode_Dict[a][1])
    for c in consequents[0]:
        conseq_list.append(Alias2StockCode_Dict[c][1])
    print(antec_list, '-->', conseq_list, consequents[1])

This is the end of Lab 3

**Ceni Babaoglu, PhD**

The Chang School of Continuing Education

Toronto Metropolitan University